In [1]:
#Jason Galvan

In [2]:
!pip install iexfinance
!pip install pandas
!pip install matplotlib
!pip install gensim
!pip install pyLDAvis
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
#Importing Libraries to Clean + Tokenize the Dataset for Processing  

In [4]:
import requests, json, re
import pandas as pd
import matplotlib
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
from datetime import datetime
from iexfinance.stocks import get_historical_data

stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation) 

def text_cleanup(input_text):
    one = " ".join([i for i in input_text.lower().split() if i not in stopwords])
    two = "".join(re.sub(r'[^a-zA-Z ]', '', i) for i in one if i not in punctuation)
    three = [WordNetLemmatizer().lemmatize(i) for i in two.split()]
    return three

In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

In [6]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [7]:
#Importing Deduplicated Dataset with Text

In [8]:
json_data=open('/Users/batman/Downloads/sorted_deduplicatedData.json').readlines()
dedup_subset = []
for line in json_data:
    dedup_subset.append(json.loads(line))
print(len(dedup_subset))

2494


In [9]:
#Converting json file to Pandas Dataframe
df = pd.DataFrame(dedup_subset, columns=['id', 'title', 'text'])

In [10]:
df.head(5)

,id,title,text
0,8,"Microsoft joins Amazon, IBM in pausing face sc...",Microsoft has become the third big tech compan...
1,13,Microsoft’s Brad Smith says company will not s...,"Posted by Editor - Technology News | Jun 11, 2..."
2,14,Microsoft won’t sell police its facial-recogni...,Privacy advocates have long raised concerns th...
3,15,Helping to Accelerate the Search for Solutions...,Helping to Accelerate the Search for Solutions...
4,16,Microsoft says it won’t sell facial recognitio...,"Following the lead of Amazon and IBM, Microsof..."


In [11]:
#Processing Data for Training LDA Model (Cleaning Text for LDA Processing)

In [12]:
from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary
import pyLDAvis.gensim
import os

text = df[['text']].applymap(text_cleanup)['text']
dictionary = Dictionary(text)
dictionary.filter_extremes(no_below=10, no_above=0.8)
corpora = [dictionary.doc2bow(doc) for doc in text]

# Running and Trainign LDA model on the document term matrix.
lda_model = ldamodel.LdaModel(corpora, num_topics=8, id2word = dictionary, passes=50)

In [13]:
# Printing Topic Clusters and Key Words for LDA Training Model--Optimum Number of Topics = 8 (after repeatedly testing)

In [14]:
#Output format:  (Cluster + Weightage for each Keyword).  Weights are how are how important a keyword is to that topic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       t
pprint(lda_model.print_topics())
doc_lda = lda_model[corpora]

[(0,
  '0.015*"business" + 0.011*"ai" + 0.010*"data" + 0.008*"company" + '
  '0.008*"new" + 0.008*"technology" + 0.008*"platform" + 0.007*"solution" + '
  '0.006*"security" + 0.006*"said"'),
 (1,
  '0.034*"share" + 0.026*"stock" + 0.025*"company" + 0.018*"quarter" + '
  '0.017*"rating" + 0.015*"news" + 0.014*"th" + 0.012*"price" + '
  '0.010*"average" + 0.010*"research"'),
 (2,
  '0.075*"market" + 0.027*"report" + 0.020*"global" + 0.017*"analysis" + '
  '0.012*"research" + 0.011*"growth" + 0.011*"industry" + 0.010*"software" + '
  '0.010*"data" + 0.010*"key"'),
 (3,
  '0.060*"table" + 0.040*"market" + 0.033*"u" + 0.031*"revenue" + '
  '0.029*"million" + 0.026*"business" + 0.024*"size" + 0.022*"management" + '
  '0.019*"development" + 0.019*"global"'),
 (4,
  '0.041*"cloud" + 0.025*"service" + 0.015*"data" + 0.014*"network" + '
  '0.011*"u" + 0.010*"company" + 0.009*"customer" + 0.009*"system" + '
  '0.008*"solution" + 0.008*"storage"'),
 (5,
  '0.046*"price" + 0.044*"holding" + 0.040*"

In [15]:
# Evaluating LDA Model:  Results:  Good Perplexity and Coherence Score 

In [19]:
#Saving 10 articles from Deduplicated Dataset in Order to Test LDA Model 

In [16]:
# Compute Perplexity Score = Lower is Better
print('\nPerplexity Score: ', lda_model.log_perplexity(corpora))

# Compute Coherence Score = Higer is Better
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=text, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity Score:  -6.955768990481339

Coherence Score:  0.6110592016479903


In [17]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpora, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.150274 -0.145219       1        1  21.141693
0     -0.146019 -0.062556       2        1  15.423119
6     -0.136016 -0.170403       3        1  13.777663
2     -0.061545  0.173821       4        1  13.342622
5      0.374921 -0.063391       5        1  13.307036
1      0.205954 -0.097253       6        1   8.050066
4     -0.106622  0.053836       7        1   7.556013
3      0.019602  0.311165       8        1   7.401785, topic_info=             Term          Freq         Total Category  logprob  loglift
63         market  12006.000000  12006.000000  Default  30.0000  30.0000
2011        table   3956.000000   3956.000000  Default  29.0000  29.0000
2200      holding   5334.000000   5334.000000  Default  28.0000  28.0000
1996        price   6368.000000   6368.000000  Default  27.0000  27.0000
1235          inc   5262.000000   5262.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
341          data    986.331970   5695.227539   Topic8  -4.1560   0.8501
1316  application    672.474731   2597.440186   Topic8  -4.5390   1.2521
964       america    484.133606   1302.448486   Topic8  -4.8676   1.6138
1059     security    524.513000   2393.690918   Topic8  -4.7875   1.0853
442       service    566.089478   4424.901855   Topic8  -4.7112   0.5472

[637 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
3476      5  0.997265        aapl
4534      5  0.989191        abbv
302       2  0.841394  accelerate
302       5  0.005925  accelerate
302       7  0.148133  accelerate
...     ...       ...         ...
300       8  0.026573        year
2666      2  0.033415         yoy
2666      4  0.144797         yoy
2666      8  0.801952         yoy
2497      6  0.994840       zacks

[1639 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 1, 7, 3, 6, 2, 5, 4])

In [18]:
new_data=[]
my_list = corpora[25:35]
for i in my_list:
    #print(i)
    new_data.append(i)

In [19]:
len(new_data)

10

In [20]:
#Converting Dataset to a Pandas Dataframe in order to Clean for LDA Processing/Testing

In [22]:
#print(new_data)
#from pandas import DataFrame
#df2 = DataFrame (new_data,columns=['text'])
#df2.head(10)

In [23]:
#Processing Test Dataset by Applying Text_Cleaup Function 

In [25]:
from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary
import pyLDAvis.gensim
import os

text2 = df2[['text']].applymap(text_cleanup)['text']

In [26]:
# Creating a New Corpus, made of previously Unseen Documents.  "Testing" new documents on established LDA Model

#Output Equals Topic Probability Distribution for each Document [(topic#, probability#)]

In [27]:
other_texts = text2
other_corpus = [dictionary.doc2bow(txt) for txt in other_texts]


Test2=[]
i= 0
x= 0
while x < 10:
    unseen_docs = other_corpus[i]
    vector = lda_model[unseen_docs]# Obtaining Topic Probability Distribution for each Document & Printing Below (topic#, probability#) 
    print(vector) 
    x= x+1
    i= i +1
    Test2.append(vector)
    

[(5, 0.15057723), (7, 0.8459622)]
[(1, 0.81024927), (2, 0.027603118), (4, 0.1265861), (5, 0.034417886)]
[(5, 0.27192932), (7, 0.7263975)]
[(7, 0.9944878)]
[(0, 0.023046507), (5, 0.14218739), (7, 0.83251333)]
[(4, 0.101574436), (5, 0.19845688), (7, 0.67151684)]
[(0, 0.015822442), (1, 0.040301085), (4, 0.010560389), (5, 0.5347847), (7, 0.39762568)]
[(0, 0.08858282), (3, 0.019789543), (5, 0.51964873), (6, 0.016128402), (7, 0.3554488)]
[(5, 0.23851037), (7, 0.75904864)]
[(3, 0.063431844), (4, 0.019023525), (5, 0.2296745), (7, 0.6822723)]


In [240]:
#End of Assignment 8